In [24]:
import os
from datetime import date
from datetime import datetime
import dateutil.parser
import pandas as pd
import numpy as np
from unidecode import unidecode
from collections import Counter
from random import choices

import sys
sys.path.append('/Users/andrew.peters/documents/fpl/')
from odds_model.utils import betting_odds_scraper as sc
from odds_model.utils import name_conversions as nc

In [25]:
# set last day of gameweek, in YYYY, MM, DD form
# will filter for data up to or equal to date
last_date_of_gw = datetime(2023, 2, 8)

In [26]:
sc.orchestrator(last_date_of_gw)

Retrieveed 10 total matches.


[WDM] - Downloading: 100%|██████████| 8.72M/8.72M [00:00<00:00, 25.7MB/s]


Scraping attempt 1
Match 0 of 10:
['Chelsea', 'Fulham']: Fri 3 Feb
** collecting score data
** done
** collecting Anytime Assist data
** done
** collecting Anytime Goalscorer data
** done
Match 1 of 10:
['Everton', 'Arsenal']: Sat 4 Feb
** collecting score data
** done
** collecting Anytime Assist data
** done
** collecting Anytime Goalscorer data
** done
Match 2 of 10:
['Aston Villa', 'Leicester']: Sat 4 Feb
** collecting score data
** done
** collecting Anytime Assist data
** done
** collecting Anytime Goalscorer data
** done
Match 3 of 10:
['Brentford', 'Southampton']: Sat 4 Feb
** collecting score data
** done
** collecting Anytime Assist data
** done
** collecting Anytime Goalscorer data
** done
Match 4 of 10:
['Brighton', 'Bournemouth']: Sat 4 Feb
** collecting score data
** done
** collecting Anytime Assist data
** done
** collecting Anytime Goalscorer data
** done
Match 5 of 10:
['Man Utd', 'Crystal Palace']: Sat 4 Feb
** collecting score data
** done
** collecting Anytime Assi

In [27]:
sc.final_odds_df_builder()

In [28]:
fpl_data = pd.read_pickle('../data/current_fpl_costs.pkl')
ga_odds = pd.read_pickle('../data/goals_assists_odds.pkl')

In [29]:
# remove accents on player names for more consisten merges
fpl_data['name'] = fpl_data.name.apply(lambda x: unidecode(x))
ga_odds['player'] = ga_odds.player.apply(lambda x: unidecode(x))

# replace names with names from the conversion dictioary so they can merge
ga_odds['player'] = ga_odds.player.replace(nc.name_conversion_dict)
ga_odds = ga_odds.rename(columns={'player': 'name'})

In [30]:
# look for mismatches in player names: players that exist in the odds data, but don't have a matching name in the fpl data
name_mismatches = [p for p in ga_odds.name.unique() if not p in(fpl_data.name.unique())]
name_mismatches

['Cafu',
 'Gustavo Scarpa',
 'Lewis OBrien',
 'Alex Moreno',
 'Matheus Cunha',
 'Marquinhos',
 'Wouter Weghorst',
 'Carlos Vinicius']

In [31]:
last_name_mismatches = [x.split(' ')[-1] for x in name_mismatches]

In [32]:
for idx, player in enumerate(name_mismatches):
    print(f'Possible Matches for {player}')
    snippet = fpl_data[fpl_data.name.str.contains(last_name_mismatches[idx])].head()
    for name in snippet.name.unique():
        print(f"'{player}': '{name}',")    
    display(snippet)

Possible Matches for Cafu


,name,web_name,team,pos,now_cost,minutes


Possible Matches for Gustavo Scarpa
'Gustavo Scarpa': 'Gustavo Henrique Furtado Scarpa',


,name,web_name,team,pos,now_cost,minutes
590,Gustavo Henrique Furtado Scarpa,Scarpa,Nottm Forest,3,50,103


Possible Matches for Lewis OBrien


...

,name,web_name,team,pos,now_cost,minutes


Possible Matches for Wouter Weghorst
'Wouter Weghorst': 'Wout Weghorst',


,name,web_name,team,pos,now_cost,minutes
512,Wout Weghorst,Weghorst,Man Utd,4,60,158


Possible Matches for Carlos Vinicius
'Carlos Vinicius': 'Carlos Vinicius Alves Morais',


,name,web_name,team,pos,now_cost,minutes
332,Carlos Vinicius Alves Morais,Vinícius,Fulham,4,54,422


In [33]:
df = fpl_data.merge(ga_odds[['name', 'match_date', 'proba_assist', 'proba_goal']], on='name')

In [34]:
df[['assist_share', 'goal_share']] = df[['proba_assist', 'proba_goal']] / df.groupby('team')[['proba_assist', 'proba_goal']].transform(sum)

In [35]:
scorelines = pd.read_pickle('../data/scoreline_probabilities.pkl')
scorelines.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720 entries, 0 to 55
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   team        720 non-null    object        
 1   gf          720 non-null    int64         
 2   ga          720 non-null    int64         
 3   match_date  720 non-null    datetime64[ns]
 4   odds        720 non-null    float64       
 5   at_home     720 non-null    int64         
 6   proba       720 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 45.0+ KB


In [36]:
#make sure team names align in both dataframes, since that is what will be used for matching
print([t for t in df.team.unique() if not t in(scorelines.team.unique())])
print([t for t in scorelines.team.unique() if not t in(df.team.unique())])
print(len(scorelines))

[]
[]
720


In [37]:
clean_sheet_proba = scorelines.groupby(['team', 'ga'])['proba'].sum().reset_index()
clean_sheet_proba = (clean_sheet_proba[clean_sheet_proba['ga'] == 0]
                     .drop(columns='ga')
                     .rename(columns={'proba': 'clean_sheet_proba'}))

In [38]:
def simulate_results(team, num_simulations = 10000):
    possible_scores = list(zip(scorelines[scorelines['team'] == team].gf, scorelines[scorelines['team'] == team].ga))
    scoreline_weights = scorelines[scorelines['team'] == team].proba
    
    names = list(df[df.team == team].name)
    assist_weights = list(df[df.team == team].assist_share)
    goal_weights = list(df[df.team == team].goal_share)
    
    sim_goal_list = []
    sim_assist_list = []
    sim_goals_against_list = []
    sim_name_list = names * num_simulations
    round_list = []
    
    for sim in range(num_simulations):
        score = choices(possible_scores, scoreline_weights)[0]
        
        game_goal_list = []
        game_assist_list = []
        
        for goals_for in range(score[0]):
            game_goal_list.append(choices(names, goal_weights)[0])
            game_assist_list.append(choices(names, assist_weights)[0])
        
        sim_goal_list.extend([Counter(game_goal_list)[p] for p in names])
        sim_assist_list.extend([Counter(game_assist_list)[p] for p in names])
        sim_goals_against_list.extend([score[1]] * len(names))
        round_list.extend([sim] * len(names))
    
    # Make DataFrame of Results
    results_df = pd.DataFrame(zip(sim_name_list, round_list, sim_goal_list, sim_assist_list, sim_goals_against_list),
                              columns=['name', 'round', 'goals', 'assists', 'goals_against'])
    return results_df
    
    

In [39]:
all_simulations = pd.DataFrame()
for team in scorelines.team.unique():
    team_result = simulate_results(team)
    all_simulations = pd.concat([all_simulations, team_result])

In [40]:
date_path = date.today().strftime('%Y_%m_%d')
path = f'../data/historical/{date_path}'

# save to historical folder as well
all_simulations.to_pickle(f'{path}/all_simulations.pkl')

In [41]:
player_averages = (all_simulations.groupby('name')[['assists', 'goals', 'goals_against']]
                   .mean()
                   .reset_index()
                   .sort_values('goals', ascending=False)
                   .rename(columns={'goals': 'mean_goals', 'assists': 'mean_assists', 
                                    'goals_against': 'mean_goals_against'})
)

In [42]:
df = df.merge(player_averages, on='name')

In [43]:
df.head()

,name,web_name,team,pos,now_cost,minutes,match_date,proba_assist,proba_goal,assist_share,goal_share,mean_assists,mean_goals,mean_goals_against
0,Granit Xhaka,Xhaka,Arsenal,3,50,1616,2022-01-22,0.200000,0.166667,0.065663,0.057509,0.1211,0.1015,1.293
1,Mohamed Elneny,Elneny,Arsenal,3,41,111,2022-01-22,0.111111,0.118871,0.036479,0.041017,0.0659,0.0761,1.293
2,Rob Holding,Holding,Arsenal,2,42,16,2022-01-22,0.058824,0.070796,0.019313,0.024429,0.0341,0.0435,1.293
3,Thomas Partey,Partey,Arsenal,3,47,1392,2022-01-22,0.162162,0.133333,0.053240,0.046007,0.0935,0.0845,1.293
4,Martin Odegaard,Ødegaard,Arsenal,3,69,1522,2022-01-22,0.337553,0.294388,0.110823,0.101580,0.1974,0.1867,1.293


In [44]:
# Goalkeepers aren't in the dataframe above, since they don't have any goals/assists betting from the bookkeepers. 
# Add them in.

goalkeepers = fpl_data[fpl_data.pos == 1]
goalkeepers = goalkeepers.merge(df[['team', 'match_date', 'mean_goals_against']].drop_duplicates(), on='team')
df = pd.concat([df, goalkeepers])

In [45]:
df = df.merge(clean_sheet_proba, on='team')

In [46]:
df.to_pickle('../data/simulated_player_outcomes.pkl')

date_path = date.today().strftime('%Y_%m_%d')
path = f'../data/historical/{date_path}'

# save to historical folder as well
df.to_pickle(f'{path}/simulated_player_outcomes.pkl')